In [1]:
import sys
import os 
import pandas as pd
import numpy as np

import soepy

from dev_library import plot_basics_choices
from dev_library import plot_basics_wages
from dev_library import df_alignment

import pandas as pd
pd.set_option('display.max_rows', 500)
sys.path.insert(0, os.environ["PROJECT_DIR"] + "/estimations/peisenha_exploration")
os.chdir(os.environ["PROJECT_DIR"] + "/estimations/peisenha_exploration")


fname = os.environ["PROJECT_DIR"] + "/resources/soepcore_struct_prep.dta"
df_obs = pd.read_stata(fname, convert_categoricals = False)
df_obs = df_alignment(df_obs, is_obs=True)

In [3]:
fname = os.environ["PROJECT_DIR"] + "/estimations/peisenha_exploration"
params_start  = pd.read_pickle(fname + "/start.soepy.pkl") 

In [4]:
df_sim_base = soepy.simulate(params_start, "resources/model_spec_init.yml")
